In [1]:
import pandas as pd
import numpy as np
import pandas_profiling

from datetime import datetime, timedelta

# Functions

In [2]:
data_path = './data/'

def read_data(fn):
    df = pd.read_csv(data_path+fn)
    print(df.head())
    return df

def save_tables(df, fn):
    df.to_csv(data_path+fn, index=False)

# View data

In [3]:
!ls data/

agenteSalas_cantidad_salas.csv	ISIS_equipos.csv.xlsx
agenteSalas_equipos.csv		ISIS_historial.csv
agenteSalas_historial.csv	ISIS_historial.csv.xlsx
ips_salas.txt			tabla_equipo.csv
ISIS_cantidad_salas.csv		tabla_sala.csv
ISIS_equipos.csv		tabla_usuario.csv


In [4]:
df_cantidad_salas = read_data('agenteSalas_cantidad_salas.csv')

      sala  total
0  waira 1     47
1  waira 2     30
2   turing     39


In [5]:
df_equipos = read_data('agenteSalas_equipos.csv')

   equipo     DireccionIP  usuario
0  ISC203  157.253.202.13  no user
1  ISC301  157.253.239.11  no user
2  ISC302  157.253.239.12  no user
3  ISC202  157.253.202.12  no user
4  ISC201  157.253.202.11  no user


In [6]:
df_historial = read_data('agenteSalas_historial.csv')

   usuario  equipo                fecha
0  no user  ISC419  2017-04-04 09:07:33
1  no user  ISC419  2017-04-18 09:14:05
2  no user  ISC419  2017-04-18 11:04:54
3  no user  ISC419  2017-05-08 12:03:14
4  no user  ISC419  2017-05-09 13:17:00


# Dimension: Sala

### id (PK) | nombre | num_equipos

In [7]:
df_cantidad_salas.columns = ['nombre', 'num_equipos']
df_cantidad_salas['id'] = df_cantidad_salas.index

In [8]:
col_loc_sala = df_cantidad_salas.columns.get_loc('nombre')

for i in range(len(df_cantidad_salas)):
    nombre = df_cantidad_salas.iloc[i, col_loc_sala]
    df_cantidad_salas.iloc[i, col_loc_sala] = nombre.replace(' ', '')

In [9]:
df_cantidad_salas = df_cantidad_salas[['id', 'nombre', 'num_equipos']]

In [10]:
df_cantidad_salas

,id,nombre,num_equipos
0,0,waira1,47
1,1,waira2,30
2,2,turing,39


In [11]:
save_tables(df_cantidad_salas,'tabla_sala.csv')

# Dimension: Equipo
### id (PK) | nombre | ip 

In [12]:
del df_equipos['usuario']
df_equipos.columns = ['nombre', 'ip']
df_equipos['id'] = df_equipos.index

In [13]:
df_equipos = df_equipos[['id', 'nombre', 'ip']]

In [14]:
df_equipos.head()

,id,nombre,ip
0,0,ISC203,157.253.202.13
1,1,ISC301,157.253.239.11
2,2,ISC302,157.253.239.12
3,3,ISC202,157.253.202.12
4,4,ISC201,157.253.202.11


In [15]:
save_tables(df_equipos,'tabla_equipo.csv')

# Dimension: Usuario
### id (PK) | login

In [16]:
usuarios = np.array( df_historial['usuario'].value_counts().keys(), dtype='str' )

In [17]:
df_usuarios = pd.DataFrame(usuarios, columns=['login'])

In [18]:
df_usuarios = df_usuarios.drop(df_usuarios.index[0])

In [19]:
df_usuarios['id'] = df_usuarios.index

In [20]:
df_usuarios = df_usuarios[['id', 'login']]

In [21]:
df_usuarios.head()

,id,login
1,1,sisnd.munoz10
2,2,isc215administrador
3,3,sisexamen
4,4,sisturing
5,5,isc201administrador


In [22]:
save_tables(df_usuarios, 'tabla_usuario.csv')

# Dimension: Fecha y Fact Table: Uso Sala
### id (PK) | anio | mes | dia | hora | minuto
### id (PK) | id_usuario (FK) | id_equipo (FK) | id_fecha_inicio (FK) | id_fecha_fin (FK) | id_sala (FK) | duracion(min) 

In [23]:
df_historial = df_historial.sort_values(by=['equipo', 'fecha'])

In [24]:
df_historial.head()

,usuario,equipo,fecha
85037,isc201administrador,ISC201,2016-01-12 07:25:55
85113,no user,ISC201,2016-01-12 08:01:42
85114,isc201administrador,ISC201,2016-01-12 08:01:56
85116,no user,ISC201,2016-01-12 08:02:22
85117,isc201administrador,ISC201,2016-01-12 08:02:26


In [25]:
df_fechas = pd.DataFrame(columns=['id', 'anio', 'mes', 'dia', 'hora', 'minuto'])
df_fact = pd.DataFrame(columns=['id', 'id_usuario', 'id_equipo', 'id_fecha_inicio', 'id_fecha_fin', 
                                'id_sala', 'duracion'])

In [26]:
id_waira1 = int( df_cantidad_salas['id'][df_cantidad_salas['nombre'] == 'waira1'] )
id_waira2 = int( df_cantidad_salas['id'][df_cantidad_salas['nombre'] == 'waira2'] )
id_turing = int( df_cantidad_salas['id'][df_cantidad_salas['nombre'] == 'turing'] )

def dar_id_sala(equipo):
    if "ISC3" in equipo: 
        return id_waira1
    elif "ISC2" in equipo: 
        return id_waira2
    elif "ISC4" in equipo: 
        return id_turing
    
def dar_id_equipo(eq):
    return int( df_equipos[ df_equipos['nombre'] == eq ]['id'] )

def dar_id_usuario(us):
    return int( df_usuarios[ df_usuarios['login'] == us ]['id'] )

In [27]:
col_loc_usuario = df_historial.columns.get_loc('usuario')
col_loc_equipo = df_historial.columns.get_loc('equipo')
col_loc_fecha = df_historial.columns.get_loc('fecha')

equipos = df_equipos['nombre']
cont_fecha = 0
cont_fact = 0

for eq in equipos:
    indices = np.where( df_historial['equipo'] == eq )[0]
    df_subset = df_historial.loc[df_historial['equipo'] == eq]
    
    if df_subset.iloc[0, col_loc_usuario] == 'no user':
        df_subset = df_subset.drop(df_subset.index[0])
    
    for i in range(len(df_subset)//2):
        usuario = df_subset.iloc[2*i, col_loc_usuario]
        no_user = df_subset.iloc[2*i+1, col_loc_usuario]

        fecha_inicio = df_subset.iloc[2*i, col_loc_fecha]
        fecha_fin = df_subset.iloc[2*i+1, col_loc_fecha]
        
        fecha_inicio_date = datetime.strptime(fecha_inicio, '%Y-%m-%d %H:%M:%S')
        fecha_fin_date = datetime.strptime(fecha_fin, '%Y-%m-%d %H:%M:%S')
        
        dict_fecha_inicio = {'id':cont_fecha, 'anio':fecha_inicio_date.year, 'mes':fecha_inicio_date.month, 
                             'dia':fecha_inicio_date.day, 'hora':fecha_inicio_date.hour, 
                             'minuto':fecha_inicio_date.minute}
        dict_fecha_fin = {'id': cont_fecha+1, 'anio':fecha_fin_date.year, 'mes':fecha_fin_date.month, 
                          'dia':fecha_fin_date.day, 'hora':fecha_fin_date.hour, 
                          'minuto':fecha_fin_date.minute}
        
        df_fechas = df_fechas.append(dict_fecha_inicio, ignore_index=True)
        df_fechas = df_fechas.append(dict_fecha_fin, ignore_index=True)
        
        duracion = (fecha_fin_date - fecha_inicio_date).seconds / 60 #min
        
        dict_fact = {'id': cont_fact+1, 'id_usuario':dar_id_usuario(usuario), 'id_equipo':dar_id_equipo(eq), 
                     'id_fecha_inicio':cont_fecha, 'id_fecha_fin':cont_fecha+1, 'id_sala':dar_id_sala(eq), 
                     'duracion':duracion}
        
        df_fact = df_fact.append(dict_fact, ignore_index=True)

        cont_fecha += 2
        cont_fact += 1

In [36]:
df_fechas.head()

,id,anio,mes,dia,hora,minuto
0,0,2016,1,18,0,1
1,1,2016,1,18,0,33
2,2,2016,1,18,1,30
3,3,2016,1,18,1,54
4,4,2016,1,18,2,5


In [39]:
df_fact.id = df_fact.id.astype(int)
df_fact.id_usuario = df_fact.id_usuario.astype(int)
df_fact.id_equipo = df_fact.id_equipo.astype(int)
df_fact.id_fecha_inicio = df_fact.id_fecha_inicio.astype(int)
df_fact.id_fecha_fin = df_fact.id_fecha_fin.astype(int)
df_fact.id_sala = df_fact.id_sala.astype(int)
df_fact.duracion = df_fact.duracion.astype(int)

In [40]:
df_fact.head()

,id,id_usuario,id_equipo,id_fecha_inicio,id_fecha_fin,id_sala,duracion
0,1,55,0,0,1,1,32
1,2,107,0,2,3,1,24
2,3,107,0,4,5,1,4
3,4,398,0,6,7,1,3
4,5,20,0,8,9,1,1


In [38]:
save_tables(df_fechas,'tabla_fecha.csv')

In [41]:
save_tables(df_fact,'tabla_fact.csv')

# Profiling

## Dimension: Sala

In [42]:
pandas_profiling.ProfileReport(df_cantidad_salas)

Number of variables,3
Number of observations,3
Total Missing (%),0.0%
Total size in memory,152.0 B
Average record size in memory,50.7 B
Numeric,2
Categorical,0
Date,0
Text (Unique),1
Rejected,0
Distinct count,3


## Dimension: Equipo

In [43]:
pandas_profiling.ProfileReport(df_equipos)

Number of variables,3
Number of observations,116
Total Missing (%),0.0%
Total size in memory,2.8 KiB
Average record size in memory,24.7 B
Numeric,1
Categorical,0
Date,0
Text (Unique),2
Rejected,0
Distinct count,116


## Dimension: Usuario

In [44]:
pandas_profiling.ProfileReport(df_usuarios)

Number of variables,3
Number of observations,1379
Total Missing (%),0.0%
Total size in memory,32.4 KiB
Average record size in memory,24.1 B
Numeric,1
Categorical,0
Date,0
Text (Unique),1
Rejected,1
Correlation,1


## Dimension: Fecha

In [45]:
pandas_profiling.ProfileReport(df_fechas)

Number of variables,6
Number of observations,251478
Total Missing (%),0.0%
Total size in memory,11.5 MiB
Average record size in memory,48.0 B
Numeric,6
Categorical,0
Date,0
Text (Unique),0
Rejected,0
Distinct count,2


## Fact Table

In [46]:
pandas_profiling.ProfileReport(df_fact)

Number of variables,7
Number of observations,125739
Total Missing (%),0.0%
Total size in memory,6.7 MiB
Average record size in memory,56.0 B
Numeric,4
Categorical,0
Date,0
Text (Unique),0
Rejected,3
Distinct count,1137
